In [1]:
import librosa
from librosa import display
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import os
import math
import gdown
import soundfile as sf
import glob
from IPython.display import Image

In [3]:
def normalisation(x, y):
    # Find RMSx (i.e. the target RMS)
    rms_x = np.sqrt(np.mean(x**2))
    # Normalize audio
    #  Find scaling factor gamma
    N = x.shape[0]
    gamma = np.sqrt((N*rms_x**2)/(np.sum(y**2)))
    # print(f'Scaling factor :{gamma:.4f}')
    y_norm = gamma * y
    return y_norm

In [1]:
# Note: Speech and background signals must have the same length in order to be summed together. This is never the case.
# If the background signal is shorter, we repeat it until reaching the length of the speech signal.
# If the speech signal is shorter, we just truncate the background signal to the length of the speech signal.

# ***********************************************************************************************
# 1. Add noise to a clean speech signal using SNR of 2 dB
def add_noise(clean_speech_filename, x, noise_filename, snr_db, sampling_rate, output_dir, clipTime):
      s, _ = librosa.load(noise_filename, sr=sampling_rate, mono=True)
      x = x[sampling_rate:sampling_rate*clipTime]
      # First let's match the length of the two signals
      x_len = x.shape[0]
      s_len = s.shape[0]
    
      if x_len > s_len:
        # Calculate how many times we need to repeat the signal and round up to the nearest integer
        reps = math.ceil(x_len/s_len)
    
        # Use the function np.tile to repeat an array
        s = np.tile(s, reps)
    
      # Truncate the background signal
      s = s[:x_len]
    
      # Check if the lengths are the same
      assert x_len == s.shape[0]
    
      # Convert SNRdb to linear
      snr = 10**(snr_db/10)
    
      # Estimate noise and signal power
      sp = np.sqrt(np.mean(s**2))
    
      xp = np.sqrt(np.mean(x**2))
    
      # Calculate desired noise power based on the SNR value
      # P^ noise = P singal / SNR
      sp_target = xp/snr
    
      # Scale factor noise signal
      # α = P^ noise / P noise
      alpha = sp_target/sp
      # print(f'alpha={alpha}')
    
      # Sum speech and background noise
      # y[n]=x[n]+α⋅s[n]
      y = x + alpha*s
      y_norm = normalisation(x, y)

      # Ensure output directory exists
      if not os.path.exists(output_dir):
            os.makedirs(output_dir)
    
      # Determine the output filename by adding '_Rev' before the file extension
      base, ext = os.path.splitext(os.path.basename(clean_speech_filename))
      output_filename = f"{base}_Noised{ext}"
      full_output_path = os.path.join(output_dir, output_filename)
      # Saving the reverberant audio to a .wav file
      sf.write(full_output_path, y_norm, samplerate=sampling_rate)
      return y_norm

# Load noise
# noise_filename = './Noise/boeing.wav'
# clean_speech_filename = './C_01_CHOP_FA.wav'
# x, sampling_rate = librosa.load(clean_speech_filename, mono=True, sr=8000)

# Mix speech and noise at 5 dB
# snr_db = 2
# clipTime = 5  # Duration in seconds to clip and process
# output_dir = './'
# y = add_noise(clean_speech_filename, x, noise_filename, snr_db, sampling_rate, output_dir, clipTime)
# sf.write(full_output_path, y, samplerate=sampling_rate)


# Player
# ipd.display(ipd.Audio(x, rate=sampling_rate, normalize=False))
# ipd.display(ipd.Audio(y, rate=sampling_rate, normalize=False))

In [45]:
def process_folder(folder_path, noise_filename, clipTime, output_dir):
    # Search for all .wav files in the folder
    wav_files = glob.glob(os.path.join(folder_path, '*.wav'))
    snr_db = 2
    for wav_file in wav_files:
        x, sampling_rate = librosa.load(wav_file, mono=True, sr=8000)
        # print(f"Processing {wav_file}...")
        add_noise(wav_file, x, noise_filename, snr_db, sampling_rate, output_dir, clipTime)
# clip_time = 4  # Duration in seconds to clip and process
# process_folder(input_folder, noise_file, clip_time, output_folder)

In [ ]:
# ================================ echo ================================

input_folder = './TCD-VOIP/Test Set/echo/ref'  # Folder with original .wav files
output_folder = './Noised_Audio/echo_sports'  # Folder to save processed .wav files
noise_file = './Noise/sports.wav'  # Room Impulse Response file
clip_time = 4  # Duration in seconds to clip and process
process_folder(input_folder, noise_file, clip_time, output_folder)

In [ ]:
input_folder = './TCD-VOIP/Test Set/echo/ref'  # Folder with original .wav files
output_folder = './Noised_Audio/echo_boeing'  # Folder to save processed .wav files
noise_file = './Noise/boeing.wav'  # Room Impulse Response file
clip_time = 4  # Duration in seconds to clip and process
process_folder(input_folder, noise_file, clip_time, output_folder)

In [ ]:
input_folder = './TCD-VOIP/Test Set/echo/ref'  # Folder with original .wav files
output_folder = './Noised_Audio/echo_crowded'  # Folder to save processed .wav files
noise_file = './Noise/crowded.wav'  # Room Impulse Response file
clip_time = 4  # Duration in seconds to clip and process
process_folder(input_folder, noise_file, clip_time, output_folder)

In [ ]:
input_folder = './TCD-VOIP/Test Set/echo/ref'  # Folder with original .wav files
output_folder = './Noised_Audio/echo_machines'  # Folder to save processed .wav files
noise_file = './Noise/machines.wav'  # Room Impulse Response file
clip_time = 4  # Duration in seconds to clip and process
process_folder(input_folder, noise_file, clip_time, output_folder)

In [ ]:
# ================================ compspkr ================================

input_folder = './TCD-VOIP/Test Set/compspkr/ref'  # Folder with original .wav files
output_folder = './Noised_Audio/compspkr_sports'  # Folder to save processed .wav files
noise_file = './Noise/sports.wav'  # Room Impulse Response file
clip_time = 4  # Duration in seconds to clip and process
process_folder(input_folder, noise_file, clip_time, output_folder)

In [ ]:
input_folder = './TCD-VOIP/Test Set/compspkr/ref'  # Folder with original .wav files
output_folder = './Noised_Audio/compspkr_boeing'  # Folder to save processed .wav files
noise_file = './Noise/boeing.wav'  # Room Impulse Response file
clip_time = 4  # Duration in seconds to clip and process
process_folder(input_folder, noise_file, clip_time, output_folder)

In [ ]:
input_folder = './TCD-VOIP/Test Set/compspkr/ref'  # Folder with original .wav files
output_folder = './Noised_Audio/compspkr_crowded'  # Folder to save processed .wav files
noise_file = './Noise/crowded.wav'  # Room Impulse Response file
clip_time = 4  # Duration in seconds to clip and process
process_folder(input_folder, noise_file, clip_time, output_folder)

In [ ]:
input_folder = './TCD-VOIP/Test Set/compspkr/ref'  # Folder with original .wav files
output_folder = './Noised_Audio/compspkr_machines'  # Folder to save processed .wav files
noise_file = './Noise/machines.wav'  # Room Impulse Response file
clip_time = 4  # Duration in seconds to clip and process
process_folder(input_folder, noise_file, clip_time, output_folder)

In [ ]:
# ================================ clip ================================

input_folder = './TCD-VOIP/Test Set/clip/ref'  # Folder with original .wav files
output_folder = './Noised_Audio/clip_sports'  # Folder to save processed .wav files
noise_file = './Noise/sports.wav'  # Room Impulse Response file
clip_time = 4  # Duration in seconds to clip and process
process_folder(input_folder, noise_file, clip_time, output_folder)

In [ ]:
input_folder = './TCD-VOIP/Test Set/clip/ref'  # Folder with original .wav files
output_folder = './Noised_Audio/clip_boeing'  # Folder to save processed .wav files
noise_file = './Noise/boeing.wav'  # Room Impulse Response file
clip_time = 4  # Duration in seconds to clip and process
process_folder(input_folder, noise_file, clip_time, output_folder)

In [ ]:
input_folder = './TCD-VOIP/Test Set/clip/ref'  # Folder with original .wav files
output_folder = './Noised_Audio/clip_crowded'  # Folder to save processed .wav files
noise_file = './Noise/crowded.wav'  # Room Impulse Response file
clip_time = 4  # Duration in seconds to clip and process
process_folder(input_folder, noise_file, clip_time, output_folder)

In [ ]:
input_folder = './TCD-VOIP/Test Set/clip/ref'  # Folder with original .wav files
output_folder = './Noised_Audio/clip_machines'  # Folder to save processed .wav files
noise_file = './Noise/machines.wav'  # Room Impulse Response file
clip_time = 4  # Duration in seconds to clip and process
process_folder(input_folder, noise_file, clip_time, output_folder)

In [ ]:
# ================================ noise ================================

input_folder = './TCD-VOIP/Test Set/noise/ref'  # Folder with original .wav files
output_folder = './Noised_Audio/noise_sports'  # Folder to save processed .wav files
noise_file = './Noise/sports.wav'  # Room Impulse Response file
clip_time = 4  # Duration in seconds to clip and process
process_folder(input_folder, noise_file, clip_time, output_folder)

In [ ]:
input_folder = './TCD-VOIP/Test Set/noise/ref'  # Folder with original .wav files
output_folder = './Noised_Audio/noise_boeing'  # Folder to save processed .wav files
noise_file = './Noise/boeing.wav'  # Room Impulse Response file
clip_time = 4  # Duration in seconds to clip and process
process_folder(input_folder, noise_file, clip_time, output_folder)

In [ ]:
input_folder = './TCD-VOIP/Test Set/noise/ref'  # Folder with original .wav files
output_folder = './Noised_Audio/noise_crowded'  # Folder to save processed .wav files
noise_file = './Noise/crowded.wav'  # Room Impulse Response file
clip_time = 4  # Duration in seconds to clip and process
process_folder(input_folder, noise_file, clip_time, output_folder)

In [ ]:
input_folder = './TCD-VOIP/Test Set/noise/ref'  # Folder with original .wav files
output_folder = './Noised_Audio/noise_machines'  # Folder to save processed .wav files
noise_file = './Noise/machines.wav'  # Room Impulse Response file
clip_time = 4  # Duration in seconds to clip and process
process_folder(input_folder, noise_file, clip_time, output_folder)